# Mapping of Allen Cell Types Database, MouseLight and NeuroMorpho.Org neuron morphologies to Neuroshapes

# TODO: Add PyShacl validation

## Prerequisites

This notebook assumes you've created a project within the [demo organization](https://sandbox.bluebrainnexus.io/web/demo/) of the sandbox deployment of Blue Brain Nexus.

## Step 1: Configure the environment you will work in

### Step 1a: Configure the environment

In [ ]:
!pip install -U nexus-sdk
!pip install allensdk
!pip install rdflib
!pip install SPARQLWrapper

In [ ]:
import nexussdk as nexus
import getpass

from allensdk.core.cell_types_cache import CellTypesCache
from allensdk.api.queries.cell_types_api import CellTypesApi
from allensdk.core.cell_types_cache import ReporterStatus as RS

import utils as ut

In [ ]:
%load_ext autoreload # TODO: remove before publishing

In [ ]:
%autoreload 1 # TODO: remove before publishing

In [ ]:
%aimport utils # TODO: remove before publishing
%aimport sparqlendpointhelper # TODO: remove before publishing
%aimport dataset # TODO: remove before publishing

We will be working in the **production** environment of Blue Brain Nexus

In [ ]:
DEPLOYMENT = "https://sandbox.bluebrainnexus.io/v1"

Provide your **token** below. Your token can be obtained after log-in by clicking on *Copy token* in the top left corner of [Nexus Web](https://bbp.epfl.ch/nexus/web/)

In [ ]:
TOKEN = getpass.getpass()

Configure your environment and token to be used for the nexus python SDK

In [ ]:
nexus.config.set_environment(DEPLOYMENT)

In [ ]:
nexus.config.set_token(TOKEN)

We will be working in the **demo** organization of Blue Brain Nexus

In [ ]:
ORGANIZATION = "demo"

### Step 1b: Create your project space in Blue Brain Nexus

Projects belong to an organization and and they are used to :

* Group and categorize sub-resources.
* Define settings that apply for operations on all sub-resources.
* Provide isolation from resources inside other projects. This behavior can be changed by defining so-called resolvers.

Check out the documentation for more information: 

* https://bluebrainnexus.io/docs/api/index.html
* https://bluebrainnexus.io/docs/tutorial/getting-started/quick-start/index.html

To create a project, set a project label and description below.

In [ ]:
PROJECT_LABEL = "nmc"

In [ ]:
PROJECT_DESCRIPTION = "NMC project"

In [ ]:
# TODO: should there be API-mappings, base, vocab?

In [ ]:
response = nxs.projects.create(org_label=ORGANIZATION_LABEL, project_label=PROJECT_LABEL, description=PROJECT_DESCRIPTION)

### Step 1c: Load the schemas from Neuroshapes into your project space

## Step 2: Download neuron morphologies from public databases

### Step 2a: Download mouse and human neuron morphologies from the Allen Cell Types Database

We will be working with human and mouse neuron morphology data from the [Allen Cell Types Database](https://celltypes.brain-map.org/). The [AllenSDK](https://allensdk.readthedocs.io/en/latest/) can be used for data download

Set the cell types cache for the Allen Cell Types Database: "The CellTypesCache class provides a Python interface for downloading data in the Allen Cell Types Database into well known locations so that you don’t have to think about file names and directories."

In [ ]:
ctc = CellTypesCache(manifest_file="allen_cell_types_db/manifest.json")

#### Download neuron morphologies from human tissue:

Get all cells from the Allen Cell Types Database which are from human and have a reconstruction 

In [ ]:
human_cells = ctc.get_cells(species=[CellTypesApi.HUMAN], require_reconstruction = True)

In [ ]:
print("Total of human cells with reconstruction: %d" % len(human_cells))
print("---")
print("Metadata of an example cell (human):")
ut.pretty_print(human_cells[0])

In [ ]:
human_cellIDs = [c["id"] for c in human_cells][0:20] # TODO: remove the limit of 20

In [ ]:
human_reconstruction = [ctc.get_reconstruction(i) for i in human_cellIDs]

#### Download neuron morphologies from mouse tissue

Get all cells from the Allen Cell Types Database which are from mouse and have a reconstruction 

In [ ]:
mouse_cells = ctc.get_cells(species=[CellTypesApi.MOUSE], require_reconstruction = True)

In [ ]:
print("Total of mouse cells with reconstruction: %d" % len(mouse_cells))
print("---")
print("Metadata of an example cell (mouse):")
ut.pretty_print(mouse_cells[0])

We will download the first twenty of those neuron morphology reconstructions

In [ ]:
mouse_cellIDs = [c["id"] for c in mouse_cells][0:20] # TODO: remove the limit of 20

In [ ]:
mouse_reconstruction = [ctc.get_reconstruction(i) for i in mouse_cellIDs]

 ### Step 2a: Explore the data structure of the Allen Cell Types Database

#### The **cells.json** metadata file 

This file contains all the available metadata of all cells currently available through the Allen Cell Types Database. The structure presents one object per cell.

In [ ]:
allen_cell_types_meta = ut.get_json("allen_cell_types_db/cells.json")
print("Metadata from the cells.json file of an example cell (human):")
ut.pretty_print(allen_cell_types_meta[0])

#### The metadata exposed through the **allenSDK**

In [ ]:
print("Metadata exposed through the allenSDK of an example cell (human):")
ut.pretty_print(human_cells[0])

#### The **folder structure** of downloaded data

When downloading data from the Allen Cell Types Database through the allenSDK, a folder per neuron morphology file is created. The folder name is composed as follows: **specimen_allenID**. This folder contains the neuron morphology reconstruction with the filename **reconstruction.swc**

### Step 2b: Download  neuron morpholgies from the NeuroMorpho.Org database

### Step 2b: Explore the data structure of the neuron morphologies from NeuroMorpho.Org

### Step 2c: Access neuron morphologies downloaded from the MouseLight project

### Step 2c: Explore the data structure of the neuron morphologies from  the MouseLight project

## Step 4: Store the downloaded neuron morphology reconstruction files from the three databases in Blue Brain Nexus

#### Integrate the downloaded neuron morphology reconstruction files into Blue Brain Nexus

In [ ]:
morph_files_meta = {}

Store the downloaded human neuron morphology reconstructions in Blue Brain Nexus:

In [ ]:
for cellID in human_cellIDs:
    file_path = f"./allen_cell_types_db/specimen_{cellID}/reconstruction.swc"
    response = nexus.files.create(org_label=ORGANIZATION, project_label=PROJECTLABEL, filepath=file_path)
    morph_files_meta[cellID] = {
        "file_name": response["_filename"],
        "content_value": response["_bytes"],
        "file_id": response["@id"],
        "digest_value": response["_digest"]["_value"]}

Store the downloaded mouse neuron morphology reconstructions in Blue Brain Nexus:

In [ ]:
for cellID in mouse_cellIDs:
    file_path = f"./allen_cell_types_db/specimen_{cellID}/reconstruction.swc"
    response = nexus.files.create(org_label=ORGANIZATION, project_label=PROJECTLABEL, filepath=file_path)
    morph_files_meta[cellID] = {
        "file_name": response["_filename"],
        "content_value": response["_bytes"],
        "file_id": response["@id"],
        "digest_value": response["_digest"]["_value"]}

## Step 5: Map the metadata to Neuroshapes

## Step 6: Generate provenance-based data entities and store them in Blue Brain Nexus

In [ ]:
# TODO: Bring the relevant agents and protocols into the project space

This is the Grid identifier of the Allen Institute of Brain Science which will be used to asign contribution:

In [ ]:
ALLEN_GRID = "https://www.grid.ac/institutes/grid.417881.3"

Generate the provenance entities for **Subject**, **PatchedCell** and **NeuronMorphology** for the human neuron morphology reconstructions and store them in Blue Brain Nexus:

In [ ]:
for human_cellID in human_cellIDs:
        morph_meta = list(filter(lambda cell: cell['specimen__id'] == human_cellID, allen_cell_types_meta))[0]
        morph_file_meta = morph_files_meta[human_cellID]

        try:
            subject = ut.subject(morph_meta)
            nexus.resources.create(org_label=ORGANIZATION, project_label=PROJECTLABEL, data=subject)
            ut.pretty_print(subject)
        except nexus.HTTPError as e:
            nexus.tools.pretty_print(e.response.json())

        try:    
            patchedcell = ut.patchedcell(morph_meta, ALLEN_GRID)
            nexus.resources.create(org_label=ORGANIZATION, project_label=PROJECTLABEL, data=patchedcell)
            ut.pretty_print(patchedcell)
        except nexus.HTTPError as e:
            nexus.tools.pretty_print(e.response.json())
            
        try:
            neuronmorphology = ut.neuronmorphology(morph_meta, ALLEN_GRID, morph_file_meta)
            nexus.resources.create(org_label=ORGANIZATION, project_label=PROJECTLABEL, data=neuronmorphology)
            ut.pretty_print(neuronmorphology)
        except nexus.HTTPError as e:
            nexus.tools.pretty_print(e.response.json())

Generate the provenance entities for **Subject**, **PatchedCell** and **NeuronMorphology** for the mouse neuron morphology reconstructions and store them in Blue Brain Nexus:

In [ ]:
for mouse_cellID in mouse_cellIDs:
    morph_meta = list(filter(lambda cell: cell['specimen__id'] == mouse_cellID, allen_cell_types_meta))[0]
    morph_file_meta = morph_files_meta[mouse_cellID]

    try:
        subject = ut.subject(morph_meta)
        nexus.resources.create(org_label=ORGANIZATION, project_label=PROJECTLABEL, data=subject)
        ut.pretty_print(subject)
    except nexus.HTTPError as e:
        nexus.tools.pretty_print(e.response.json())

    try:    
        patchedcell = ut.patchedcell(morph_meta, ALLEN_GRID)
        nexus.resources.create(org_label=ORGANIZATION, project_label=PROJECTLABEL, data=patchedcell)
        ut.pretty_print(patchedcell)
    except nexus.HTTPError as e:
        nexus.tools.pretty_print(e.response.json())

    try:
        neuronmorphology = ut.neuronmorphology(morph_meta, ALLEN_GRID, morph_file_meta)
        nexus.resources.create(org_label=ORGANIZATION, project_label=PROJECTLABEL, data=neuronmorphology)
        ut.pretty_print(neuronmorphology)
    except nexus.HTTPError as e:
        nexus.tools.pretty_print(e.response.json())

Check out [Nexus Web](https://sandbox.bluebrainnexus.io/web/demo/) to view and navigate your created resources.